<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/Stocks%20Research/Trades%20for%20tomorrow/Fno_considering_3_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install pya3 nsetools

In [3]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2022, 1, 1)  # Start from a longer time frame
    to_datetime = datetime.now()  # To now
    interval = "D"  # ["1", "D"]

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Find local maxima and minima
    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]

    # Global maxima and minima
    global_maxima_index = np.argmax(close_prices)
    global_minima_index = np.argmin(close_prices)

    # Define the current price (example: last close price)
    current_price = df['close'].iloc[-1]

    # Define threshold (e.g., within 5% of the extrema)
    threshold = 0.05

    # Determine if current price is near any local or global minima or maxima
    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    # Check proximity to local minima
    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)

    # Check proximity to global minima
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    # Calculate the moving average of the volume
    df['volume_ma'] = df['volume'].rolling(window=20).mean()

    # Volume threshold (e.g., 1.5 times the average volume)
    volume_threshold = 1.5 * df['volume'].mean()

    # Enhanced signal determination using volume
    if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
        signal = "BUY"
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            support_level = close_prices[nearest_local_minima_index]
            support_volume = df.loc[nearest_local_minima_index, 'volume']
        else:
            support_level = close_prices[global_minima_index]
            support_volume = df.loc[global_minima_index, 'volume']
        if support_volume < volume_threshold:
            signal = None  # Downgrade to None if volume is not confirming
    else:
        signal = None

    return token, current_price, signal, support_level if signal == "BUY" else None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    result = analyze_stock(alice, token)
    if result[2] == "BUY":
        buy_signals.append((result[0], result[1], result[3]))

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)


Enter your user ID: 1141826
Enter your API key: 73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g
NOTE: Today's contract master file will be updated after 08:00 AM. Before 08:00 AM previous day contract file be downloaded.
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
SHRIRAMFIN-EQ   4306       585.10          559.55         
NESTLEIND-EQ    17963      2214.95         2214.45        
INDUSINDBK-EQ   5258       1043.75         1041.85        
--------------------------------------------------


Nifty 50

In [10]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import time

def get_user_credentials():
    return "1141826", "73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def analyze_stock(alice, token):
    try:
        print(f"\nAnalyzing token {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 2 years of daily data
        from_date = datetime.now() - timedelta(days=730)
        historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
        df = pd.DataFrame(historical_data).dropna()

        print(f"Data points: {len(df)}")
        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic technical indicators
        df['50_EMA'] = df['close'].ewm(span=50).mean()
        df['200_EMA'] = df['close'].ewm(span=200).mean()
        rsi = compute_rsi(df['close'])

        # Support/resistance detection
        close_prices = df['close'].values
        scaler = MinMaxScaler()
        normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

        # Dynamic window size
        window_size = max(int(len(df)*0.05), 5)
        local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]

        # Find significant support levels (last 6 months)
        valid_supports = []
        for m in local_min:
            if m < len(df)-126:  # Older than 6 months
                continue

            support_price = close_prices[m]
            current_price = close_prices[-1]

            # Price between 5% to 20% above support
            if 1.05 <= (current_price / support_price) <= 1.20:
                # Volume check (current volume > support period volume)
                if df['volume'].iloc[-1] > df['volume'].iloc[m] * 0.8:
                    valid_supports.append({
                        'price': support_price,
                        'date': df.index[m],
                        'touches': 1
                    })

        print(f"Found {len(valid_supports)} potential supports")

        if not valid_supports:
            return None

        # Cluster nearby supports
        support_clusters = []
        tolerance = np.std(close_prices)*0.3
        for sup in valid_supports:
            found = False
            for cluster in support_clusters:
                if abs(sup['price'] - cluster['price']) <= tolerance:
                    cluster['count'] += 1
                    found = True
                    break
            if not found:
                support_clusters.append({
                    'price': sup['price'],
                    'count': 1,
                    'dates': [sup['date']]
                })

        if not support_clusters:
            return None

        # Select best support cluster
        best_cluster = max(support_clusters, key=lambda x: x['count'])
        print(f"Best support: {best_cluster['price']} (Count: {best_cluster['count']})")

        # Trend filter
        if df['50_EMA'].iloc[-1] < df['200_EMA'].iloc[-1]:
            print("Failing trend filter (50EMA < 200EMA)")
            return None

        # Momentum filter
        if rsi > 65:
            print(f"RSI too high: {rsi:.1f}")
            return None

        # Risk management check
        current_price = close_prices[-1]
        distance_pct = (current_price / best_cluster['price'] - 1) * 100

        print(f"Current: {current_price}, Support: {best_cluster['price']}, Distance: {distance_pct:.1f}%")

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_cluster['price'],
            'Strength': best_cluster['count'],
            'Distance%': distance_pct,
            'RSI': rsi,
            'Trend': 'Bullish' if df['50_EMA'].iloc[-1] > df['200_EMA'].iloc[-1] else 'Bearish'
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Example tokens (Nifty 50 constituents)
    tokens = [25, 15083, 157, 236, 5900, 16669, 317, 16675, 383, 526, 10604, 547, 694, 20374, 881, 910, 1232, 7229, 1333, 467, 1348, 1363, 1394, 4963,
              1660, 5258, 1594, 11723, 1922, 11483, 2031, 10999, 11630, 17963, 2475, 14977, 2885, 21808, 4306, 3045, 3351, 11536, 3432, 3456, 3499, 13538, 3506, 1964, 11532, 3787

    ]

    signals = []
    for token in tokens:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
        time.sleep(1)  # Rate limit

    # Sort by strongest signals
    sorted_signals = sorted(signals,
        key=lambda x: (-x['Strength'], -x['Distance%']))

    print("\nTop 10 Buy Candidates:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10} {'RSI':<6} {'Trend':<10}")
    for s in sorted_signals[:10]:
        print(f"{s['Name'][:18]:<20} {s['Price']:<10.1f} {s['Support']:<10.1f} "
              f"{s['Strength']:<10} {s['Distance%']:<10.1f} {s['RSI']:<6.1f} {s['Trend']:<10}")

if __name__ == "__main__":
    main()

Session ID: {'stat': 'Ok', 'sessionID': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIyam9lOFVScGxZU3FTcDB3RDNVemVBQkgxYkpmOE4wSDRDMGVVSWhXUVAwIn0.eyJleHAiOjE3NDUzODY1NzUsImlhdCI6MTc0MDIwMjU3NSwianRpIjoiYTUxODBhMGItMWZjMi00OWVmLWJhYTctOWM3YzE1ZTgzMzU2IiwiaXNzIjoiaHR0cHM6Ly9pZGFhcy5hbGljZWJsdWVvbmxpbmUuY29tL2lkYWFzL3JlYWxtcy9BbGljZUJsdWUiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiY2U2ZTQ4MzEtZWQ3My00MGQ2LThlN2YtYWRmZDA4Y2ViMjVjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWxpY2Uta2IiLCJzaWQiOiIwZGU3Njk3MC03Zjg2LTQ3ZmEtOTVlYS0zN2Y2ZDZiY2QyODQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAyIiwiaHR0cDovL2xvY2FsaG9zdDo1MDUwIiwiaHR0cDovL2xvY2FsaG9zdDo5OTQzIiwiaHR0cDovL2xvY2FsaG9zdDo5MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtYWxpY2VibHVla2IiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFsaWNlLWtiIjp7InJvbGVzIjpbIkdVRVNUX1VTRVIiLCJBQ1RJVkVfVVNFUiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

Nifty next 50


In [16]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import time

def get_user_credentials():
    return "1141826", "73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def analyze_stock(alice, token):
    try:
        print(f"\nAnalyzing token {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 2 years of daily data
        from_date = datetime.now() - timedelta(days=730)
        historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
        df = pd.DataFrame(historical_data).dropna()

        print(f"Data points: {len(df)}")
        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic technical indicators
        df['50_EMA'] = df['close'].ewm(span=50).mean()
        df['200_EMA'] = df['close'].ewm(span=200).mean()
        rsi = compute_rsi(df['close'])

        # Support/resistance detection
        close_prices = df['close'].values
        scaler = MinMaxScaler()
        normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

        # Dynamic window size
        window_size = max(int(len(df)*0.05), 5)
        local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]

        # Find significant support levels (last 6 months)
        valid_supports = []
        for m in local_min:
            if m < len(df)-126:  # Older than 6 months
                continue

            support_price = close_prices[m]
            current_price = close_prices[-1]

            # Price between 5% to 20% above support
            if 1.05 <= (current_price / support_price) <= 1.20:
                # Volume check (current volume > support period volume)
                if df['volume'].iloc[-1] > df['volume'].iloc[m] * 0.8:
                    valid_supports.append({
                        'price': support_price,
                        'date': df.index[m],
                        'touches': 1
                    })

        print(f"Found {len(valid_supports)} potential supports")

        if not valid_supports:
            return None

        # Cluster nearby supports
        support_clusters = []
        tolerance = np.std(close_prices)*0.3
        for sup in valid_supports:
            found = False
            for cluster in support_clusters:
                if abs(sup['price'] - cluster['price']) <= tolerance:
                    cluster['count'] += 1
                    found = True
                    break
            if not found:
                support_clusters.append({
                    'price': sup['price'],
                    'count': 1,
                    'dates': [sup['date']]
                })

        if not support_clusters:
            return None

        # Select best support cluster
        best_cluster = max(support_clusters, key=lambda x: x['count'])
        print(f"Best support: {best_cluster['price']} (Count: {best_cluster['count']})")

        # Trend filter
        if df['50_EMA'].iloc[-1] < df['200_EMA'].iloc[-1]:
            print("Failing trend filter (50EMA < 200EMA)")
            return None

        # Momentum filter
        if rsi > 65:
            print(f"RSI too high: {rsi:.1f}")
            return None

        # Risk management check
        current_price = close_prices[-1]
        distance_pct = (current_price / best_cluster['price'] - 1) * 100

        print(f"Current: {current_price}, Support: {best_cluster['price']}, Distance: {distance_pct:.1f}%")

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_cluster['price'],
            'Strength': best_cluster['count'],
            'Distance%': distance_pct,
            'RSI': rsi,
            'Trend': 'Bullish' if df['50_EMA'].iloc[-1] > df['200_EMA'].iloc[-1] else 'Bearish'
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Example tokens (Nifty 50 constituents)
    tokens = [13, 10217, 3563, 17388, 6066, 1270, 19913, 305, 4668, 438, 2181, 10794, 685, 14732, 772, 10940,
              4717, 10099, 9819, 2303, 21770, 18652, 1624, 13611, 2029, 13751, 11195, 17869, 6733, 18143, 17818,
              9480, 3220, 17400, 2664, 14299, 10666, 15355, 4204, 3103, 3150, 8479, 3426, 3518, 10753, 10447, 18921, 3063, 5097, 7929

    ]

    signals = []
    for token in tokens:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
        time.sleep(1)  # Rate limit

    # Sort by strongest signals
    sorted_signals = sorted(signals,
        key=lambda x: (-x['Strength'], -x['Distance%']))

    print("\nTop 10 Buy Candidates:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10} {'RSI':<6} {'Trend':<10}")
    for s in sorted_signals[:10]:
        print(f"{s['Name'][:18]:<20} {s['Price']:<10.1f} {s['Support']:<10.1f} "
              f"{s['Strength']:<10} {s['Distance%']:<10.1f} {s['RSI']:<6.1f} {s['Trend']:<10}")

if __name__ == "__main__":
    main()

Session ID: {'stat': 'Ok', 'sessionID': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIyam9lOFVScGxZU3FTcDB3RDNVemVBQkgxYkpmOE4wSDRDMGVVSWhXUVAwIn0.eyJleHAiOjE3NDUzODY1NzUsImlhdCI6MTc0MDIwMjU3NSwianRpIjoiYTUxODBhMGItMWZjMi00OWVmLWJhYTctOWM3YzE1ZTgzMzU2IiwiaXNzIjoiaHR0cHM6Ly9pZGFhcy5hbGljZWJsdWVvbmxpbmUuY29tL2lkYWFzL3JlYWxtcy9BbGljZUJsdWUiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiY2U2ZTQ4MzEtZWQ3My00MGQ2LThlN2YtYWRmZDA4Y2ViMjVjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWxpY2Uta2IiLCJzaWQiOiIwZGU3Njk3MC03Zjg2LTQ3ZmEtOTVlYS0zN2Y2ZDZiY2QyODQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAyIiwiaHR0cDovL2xvY2FsaG9zdDo1MDUwIiwiaHR0cDovL2xvY2FsaG9zdDo5OTQzIiwiaHR0cDovL2xvY2FsaG9zdDo5MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtYWxpY2VibHVla2IiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFsaWNlLWtiIjp7InJvbGVzIjpbIkdVRVNUX1VTRVIiLCJBQ1RJVkVfVVNFUiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

Nifty 100


In [17]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import time

def get_user_credentials():
    return "1141826", "73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def analyze_stock(alice, token):
    try:
        print(f"\nAnalyzing token {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 2 years of daily data
        from_date = datetime.now() - timedelta(days=730)
        historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
        df = pd.DataFrame(historical_data).dropna()

        print(f"Data points: {len(df)}")
        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic technical indicators
        df['50_EMA'] = df['close'].ewm(span=50).mean()
        df['200_EMA'] = df['close'].ewm(span=200).mean()
        rsi = compute_rsi(df['close'])

        # Support/resistance detection
        close_prices = df['close'].values
        scaler = MinMaxScaler()
        normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

        # Dynamic window size
        window_size = max(int(len(df)*0.05), 5)
        local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]

        # Find significant support levels (last 6 months)
        valid_supports = []
        for m in local_min:
            if m < len(df)-126:  # Older than 6 months
                continue

            support_price = close_prices[m]
            current_price = close_prices[-1]

            # Price between 5% to 20% above support
            if 1.05 <= (current_price / support_price) <= 1.20:
                # Volume check (current volume > support period volume)
                if df['volume'].iloc[-1] > df['volume'].iloc[m] * 0.8:
                    valid_supports.append({
                        'price': support_price,
                        'date': df.index[m],
                        'touches': 1
                    })

        print(f"Found {len(valid_supports)} potential supports")

        if not valid_supports:
            return None

        # Cluster nearby supports
        support_clusters = []
        tolerance = np.std(close_prices)*0.3
        for sup in valid_supports:
            found = False
            for cluster in support_clusters:
                if abs(sup['price'] - cluster['price']) <= tolerance:
                    cluster['count'] += 1
                    found = True
                    break
            if not found:
                support_clusters.append({
                    'price': sup['price'],
                    'count': 1,
                    'dates': [sup['date']]
                })

        if not support_clusters:
            return None

        # Select best support cluster
        best_cluster = max(support_clusters, key=lambda x: x['count'])
        print(f"Best support: {best_cluster['price']} (Count: {best_cluster['count']})")

        # Trend filter
        if df['50_EMA'].iloc[-1] < df['200_EMA'].iloc[-1]:
            print("Failing trend filter (50EMA < 200EMA)")
            return None

        # Momentum filter
        if rsi > 65:
            print(f"RSI too high: {rsi:.1f}")
            return None

        # Risk management check
        current_price = close_prices[-1]
        distance_pct = (current_price / best_cluster['price'] - 1) * 100

        print(f"Current: {current_price}, Support: {best_cluster['price']}, Distance: {distance_pct:.1f}%")

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_cluster['price'],
            'Strength': best_cluster['count'],
            'Distance%': distance_pct,
            'RSI': rsi,
            'Trend': 'Bullish' if df['50_EMA'].iloc[-1] > df['200_EMA'].iloc[-1] else 'Bearish'
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Example tokens (Nifty 50 constituents)
    tokens = [13, 10217, 25, 3563, 15083, 17388, 6066, 1270, 157, 236, 19913, 5900, 16669, 317, 16675, 305, 4668, 383, 438, 526,
10604, 2181, 547, 10794, 685, 694, 20374, 14732, 772, 10940, 881, 910, 4717, 10099, 1232, 7229, 1333, 467, 9819, 1348,
 1363, 1394, 4963, 21770, 18652, 1660, 1624, 13611, 2029, 5258, 13751, 1594, 11195, 17869, 11723, 6733, 18143, 1922, 17818,
 11483, 9480, 3220, 2031, 10999, 17400, 11630, 17963, 2475, 2664, 14299, 14977, 10666, 15355, 2885, 21808, 4204, 3103, 4306, 3150,
 3045, 3351, 8479, 11536, 3432, 3456, 3426, 3499, 13538, 3506, 3518, 1964, 11532, 10753, 10447, 18921, 3063, 3787, 5097, 7929

    ]

    signals = []
    for token in tokens:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
        time.sleep(1)  # Rate limit

    # Sort by strongest signals
    sorted_signals = sorted(signals,
        key=lambda x: (-x['Strength'], -x['Distance%']))

    print("\nTop 10 Buy Candidates:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10} {'RSI':<6} {'Trend':<10}")
    for s in sorted_signals[:10]:
        print(f"{s['Name'][:18]:<20} {s['Price']:<10.1f} {s['Support']:<10.1f} "
              f"{s['Strength']:<10} {s['Distance%']:<10.1f} {s['RSI']:<6.1f} {s['Trend']:<10}")

if __name__ == "__main__":
    main()

Session ID: {'stat': 'Ok', 'sessionID': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIyam9lOFVScGxZU3FTcDB3RDNVemVBQkgxYkpmOE4wSDRDMGVVSWhXUVAwIn0.eyJleHAiOjE3NDUzODY1NzUsImlhdCI6MTc0MDIwMjU3NSwianRpIjoiYTUxODBhMGItMWZjMi00OWVmLWJhYTctOWM3YzE1ZTgzMzU2IiwiaXNzIjoiaHR0cHM6Ly9pZGFhcy5hbGljZWJsdWVvbmxpbmUuY29tL2lkYWFzL3JlYWxtcy9BbGljZUJsdWUiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiY2U2ZTQ4MzEtZWQ3My00MGQ2LThlN2YtYWRmZDA4Y2ViMjVjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWxpY2Uta2IiLCJzaWQiOiIwZGU3Njk3MC03Zjg2LTQ3ZmEtOTVlYS0zN2Y2ZDZiY2QyODQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAyIiwiaHR0cDovL2xvY2FsaG9zdDo1MDUwIiwiaHR0cDovL2xvY2FsaG9zdDo5OTQzIiwiaHR0cDovL2xvY2FsaG9zdDo5MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtYWxpY2VibHVla2IiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFsaWNlLWtiIjp7InJvbGVzIjpbIkdVRVNUX1VTRVIiLCJBQ1RJVkVfVVNFUiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

NIFTY 200


In [19]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import time

def get_user_credentials():
    return "1141826", "73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def analyze_stock(alice, token):
    try:
        print(f"\nAnalyzing token {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 2 years of daily data
        from_date = datetime.now() - timedelta(days=730)
        historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
        df = pd.DataFrame(historical_data).dropna()

        print(f"Data points: {len(df)}")
        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic technical indicators
        df['50_EMA'] = df['close'].ewm(span=50).mean()
        df['200_EMA'] = df['close'].ewm(span=200).mean()
        rsi = compute_rsi(df['close'])

        # Support/resistance detection
        close_prices = df['close'].values
        scaler = MinMaxScaler()
        normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

        # Dynamic window size
        window_size = max(int(len(df)*0.05), 5)
        local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]

        # Find significant support levels (last 6 months)
        valid_supports = []
        for m in local_min:
            if m < len(df)-126:  # Older than 6 months
                continue

            support_price = close_prices[m]
            current_price = close_prices[-1]

            # Price between 5% to 20% above support
            if 1.03 <= (current_price / support_price) <= 1.10:
                # Volume check (current volume > support period volume)
                if df['volume'].iloc[-1] > df['volume'].iloc[m] * 0.8:
                    valid_supports.append({
                        'price': support_price,
                        'date': df.index[m],
                        'touches': 1
                    })

        print(f"Found {len(valid_supports)} potential supports")

        if not valid_supports:
            return None

        # Cluster nearby supports
        support_clusters = []
        tolerance = np.std(close_prices)*0.3
        for sup in valid_supports:
            found = False
            for cluster in support_clusters:
                if abs(sup['price'] - cluster['price']) <= tolerance:
                    cluster['count'] += 1
                    found = True
                    break
            if not found:
                support_clusters.append({
                    'price': sup['price'],
                    'count': 1,
                    'dates': [sup['date']]
                })

        if not support_clusters:
            return None

        # Select best support cluster
        best_cluster = max(support_clusters, key=lambda x: x['count'])
        print(f"Best support: {best_cluster['price']} (Count: {best_cluster['count']})")

        # Trend filter
        if df['50_EMA'].iloc[-1] < df['200_EMA'].iloc[-1]:
            print("Failing trend filter (50EMA < 200EMA)")
            return None

        # Momentum filter
        if rsi > 65:
            print(f"RSI too high: {rsi:.1f}")
            return None

        # Risk management check
        current_price = close_prices[-1]
        distance_pct = (current_price / best_cluster['price'] - 1) * 100

        print(f"Current: {current_price}, Support: {best_cluster['price']}, Distance: {distance_pct:.1f}%")

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_cluster['price'],
            'Strength': best_cluster['count'],
            'Distance%': distance_pct,
            'RSI': rsi,
            'Trend': 'Bullish' if df['50_EMA'].iloc[-1] > df['200_EMA'].iloc[-1] else 'Bearish'
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Example tokens (Nifty 50 constituents)
    tokens = [
13, 22, 25780, 21238, 10217, 25, 3563, 15083, 17388, 6066, 21614, 30108, 11703, 1270, 157, 163, 212, 236,
14418, 275, 19913, 5900, 19585, 16669, 317, 16675, 305, 335, 2263, 4668, 4745, 11377, 2144, 383, 422, 438,
526, 10604, 23489, 11373, 2181, 547, 760, 10794, 685, 694, 20374, 21508, 11543, 15141, 4749, 1901, 14732,
772, 9599, 10940, 21690, 881, 910, 958, 676, 6545, 1023, 1008, 4717, 13528, 10099, 17875, 1232, 7229,
4244, 1333, 467, 9819, 1348, 1363, 2303, 1406, 1394, 1424, 20825, 4963, 21770, 18652, 1476, 11184, 15313,
1660, 14309, 1512, 1624, 9348, 13611, 2029, 20261, 11262, 29135, 5258, 13751, 1594, 11195, 17869, 19020,
11723, 6733, 18143, 18096, 9683, 2955, 1922, 24948, 1997, 17818, 11483, 9480, 10440, 2277, 3220, 13285,
2031, 2283, 15380, 4067, 10999, 2142, 22377, 509, 4503, 23650, 17400, 8585, 15332, 11630, 17963, 20242,
2475, 17438, 6705, 10738, 6656, 24184, 14413, 17029, 18365, 11351, 14552, 2664, 9590, 11403, 14299,
14977, 20302, 10666, 15355, 9552, 2885, 17971, 21808, 18883, 3273, 4204, 3103, 4306, 3150, 13332, 4684,3045,
2963, 3351, 3339, 3363, 12018, 8479, 3405, 3721, 11536, 3432, 3411, 3456, 3426, 3499, 20293, 13538, 3506, 3518,
               13786, 1964, 312, 11287, 11532, 10753, 10447, 18921, 3063, 14366, 3718, 3787, 11915, 5097, 7929

    ]

    signals = []
    for token in tokens:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
        time.sleep(1)  # Rate limit

    # Sort by strongest signals
    sorted_signals = sorted(signals,
        key=lambda x: (-x['Strength'], -x['Distance%']))

    print("\nTop 10 Buy Candidates:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10} {'RSI':<6} {'Trend':<10}")
    for s in sorted_signals[:10]:
        print(f"{s['Name'][:18]:<20} {s['Price']:<10.1f} {s['Support']:<10.1f} "
              f"{s['Strength']:<10} {s['Distance%']:<10.1f} {s['RSI']:<6.1f} {s['Trend']:<10}")

if __name__ == "__main__":
    main()

Session ID: {'stat': 'Ok', 'sessionID': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIyam9lOFVScGxZU3FTcDB3RDNVemVBQkgxYkpmOE4wSDRDMGVVSWhXUVAwIn0.eyJleHAiOjE3NDUzODY1NzUsImlhdCI6MTc0MDIwMjU3NSwianRpIjoiYTUxODBhMGItMWZjMi00OWVmLWJhYTctOWM3YzE1ZTgzMzU2IiwiaXNzIjoiaHR0cHM6Ly9pZGFhcy5hbGljZWJsdWVvbmxpbmUuY29tL2lkYWFzL3JlYWxtcy9BbGljZUJsdWUiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiY2U2ZTQ4MzEtZWQ3My00MGQ2LThlN2YtYWRmZDA4Y2ViMjVjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWxpY2Uta2IiLCJzaWQiOiIwZGU3Njk3MC03Zjg2LTQ3ZmEtOTVlYS0zN2Y2ZDZiY2QyODQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAyIiwiaHR0cDovL2xvY2FsaG9zdDo1MDUwIiwiaHR0cDovL2xvY2FsaG9zdDo5OTQzIiwiaHR0cDovL2xvY2FsaG9zdDo5MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtYWxpY2VibHVla2IiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFsaWNlLWtiIjp7InJvbGVzIjpbIkdVRVNUX1VTRVIiLCJBQ1RJVkVfVVNFUiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1